In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import nltk

from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from tqdm.auto import tqdm, trange
from google.colab import files
from sklearn.utils import shuffle
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# загрузим спарсенные данные
uploaded = files.upload()

Saving News.csv to News.csv
Saving NewsBisnes.csv to NewsBisnes.csv
Saving NewsCulture.csv to NewsCulture.csv
Saving NewsPolitics.csv to NewsPolitics.csv


In [70]:
sport = pd.read_csv('/content/News.csv', delimiter = ';') 
bisnes = pd.read_csv('/content/NewsBisnes.csv', delimiter = ';')
culture = pd.read_csv('/content/NewsCulture.csv', delimiter = ';') 
politics = pd.read_csv('/content/NewsPolitics.csv', delimiter = ';')

In [71]:
# Создадим одну таблицу со всеми данными
news = pd.concat([sport, bisnes, culture, politics], axis=0)

print('Shape of the `sport` table: ', sport.shape)
print('Shape of the `bisnes` table: ', bisnes.shape)
print('Shape of the `culture` table: ', culture.shape)
print('Shape of the `politics` table: ', politics.shape)

print(news.shape)

Shape of the `sport` table:  (224, 5)
Shape of the `bisnes` table:  (57, 5)
Shape of the `culture` table:  (455, 5)
Shape of the `politics` table:  (758, 5)
(1494, 5)


In [72]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1494 entries, 0 to 757
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1494 non-null   object
 1   Дата    1494 non-null   object
 2   Ссылка  1494 non-null   object
 3   Text    1494 non-null   object
 4   Topic   1494 non-null   object
dtypes: object(5)
memory usage: 70.0+ KB


In [73]:
news['Text']=news['Text']+" "+news['Title']
news.drop(['Дата','Ссылка','Title'],axis=1,inplace=True) 
news.head()

,Text,Topic
0,Сегодня в федеральных СМИ продолжилисьпубликац...,Спорт
1,«Усик обратился к IBF с просьбой немедленно от...,Спорт
2,В составе победителей отличились Артем Ильенко...,Спорт
3,Подробнее о деталях санкций МПК в разговоре с ...,Спорт
4,"Об этом сообщает«Советский спорт», ссылаясь на...",Спорт


In [74]:
# Перемешаем данные
news = shuffle(news).reset_index(drop=True) 
news.head()

,Text,Topic
0,"Звезда ""Титаника"", известный голливудский акте...",Культура
1,Сегодня в Липецке побывали члены Олимпийской с...,Спорт
2,"Французский сенат вчера, 9 апреля, одобрил нор...",Политика
3,2 августа в селе Полевые Локотцы откроется нов...,Культура
4,Сегодня Басманный суд Москвы по ходатайству сл...,Политика


In [75]:
topics = news['Topic'].unique()
print(topics, len(topics))

['Культура' 'Спорт' 'Политика' 'Бизнес'] 4


# Предобработка

In [76]:
stopwords_russian = stopwords.words('russian')

def clean_text(text): 
  lemmatizer = WordNetLemmatizer() 
  stopwords_russian = stopwords.words('russian') 
  text= re.sub('\[[^]]*\]', '', text) 
  # remove stock market tickers like $GE
  text = re.sub(r'\$\w*', '', text) 
  #removal of html tags 
  review =re.sub(r'<.*?>',' ',text) 
  # remove old style retweet text "RT" 
  text = re.sub(r'^RT[\s]+', '', text) 
  # remove hyperlinks 
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) 
  # remove hashtags 
  # only removing the hash # sign from the word 
  text = re.sub(r'#', '', text) 
  text = re.sub("["
                 u"\U0001F600-\U0001F64F" # removal of emoticons 
                 u"\U0001F300-\U0001F5FF" # symbols & pictographs 
                 u"\U0001F680-\U0001F6FF" # transport & map symbols 
                 u"\U0001F1E0-\U0001F1FF" # flags (iOS) 
                 u"\U00002702-\U000027B0" 
                 u"\U000024C2-\U0001F251" 
                 "]+",' ',text) 
  text = re.sub('[^a-zA-Z]',' ',text)
  
  text = text.lower() 
  text_tokens =word_tokenize(text) 
  
  text_clean = [] 
  for word in text_tokens: 
    if (word not in stopwords_russian and # remove stopwords
          word not in string.punctuation): # remove punctuation 
        lem_word =lemmatizer.lemmatize(word) # lemmitiging word 
        text_clean.append(lem_word) 
  text_mod=[i for i in text_clean if len(i)>2] 
  text_clean=' '.join(text_mod) 
  return text_clean

In [77]:
texts = news['Text']

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords_russian,
                                 use_idf=True, tokenizer=clean_text, ngram_range=(1,3))

In [93]:
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

In [94]:
tfidf_matrix

<1494x304 sparse matrix of type '<class 'numpy.float64'>'
	with 16930 stored elements in Compressed Sparse Row format>

In [95]:
num_clusters = 4

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, init = 'k-means++')

In [96]:
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [97]:
print(len(km.labels_))
clusters[:10]

1494


[0, 1, 3, 0, 1, 1, 1, 1, 0, 1]

In [98]:
clusterkm = km.labels_.tolist()

#k-means
out = { 'text': texts, 'cluster': clusterkm, 'topic': news['Topic'] }
frame1 = pd.DataFrame(out, columns = ['text', 'cluster', 'topic'])

In [99]:
frame1.head(10)

,text,cluster,topic
0,"Звезда ""Титаника"", известный голливудский акте...",0,Культура
1,Сегодня в Липецке побывали члены Олимпийской с...,1,Спорт
2,"Французский сенат вчера, 9 апреля, одобрил нор...",3,Политика
3,2 августа в селе Полевые Локотцы откроется нов...,0,Культура
4,Сегодня Басманный суд Москвы по ходатайству сл...,1,Политика
5,Сегодня в федеральных СМИ продолжилисьпубликац...,1,Спорт
6,Сегодня утром сотрудником почтового отделения ...,1,Культура
7,Главная военная прокуратура выявила новое злоу...,1,Политика
8,Сотрудники Госавтоинспекции Москвы в ночь на 8...,0,Культура
9,"Автомобиль, в котором находился журналист даге...",1,Политика


In [100]:
clust_res = pd.DataFrame(columns=['topic', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3'])
i = 0
for topic in topics:
    cnt = []
    for cluster in range(num_clusters):
        cnt.append(len(frame1[ frame1.topic.eq(topic) &  frame1.cluster.eq(cluster) ]))
    clust_res.loc[i] = [topic] + cnt
    i += 1
clust_res

,topic,cluster_0,cluster_1,cluster_2,cluster_3
0,Культура,168,226,45,16
1,Спорт,63,138,17,6
2,Политика,199,419,108,32
3,Бизнес,24,23,7,3


Можно предположить, что культура принадлежит 0 кластеру, а политика - 1. На счет спорта и бизнеса все не так однозначно. Возможно на такое распределение повлияло малое количество новостей данных групп.

In [101]:
# Спорт
econ_text = '''
Российский боксер Арест Саакян скончался через несколько дней после нокаута в бое на турнире «Короли нокаутов», сообщила сестра спортсмена Светлана Петросян на своей странице в Instagram.

Турнир состоялся 26 декабря в Тольятти. От полученных травм после нокаута Саакян впал в кому. У него был диагностирован отек головного мозга. Врачи несколько дней боролись за жизнь 26-летнего боксера, но он скончался, так и не придя в сознание.
'''

In [102]:
text_vec = tfidf_vectorizer.transform([econ_text])

In [103]:
km.predict(text_vec)

array([0], dtype=int32)

In [104]:
# Политика
econ_text = '''
США рассчитывают на сотрудничество с Россией по Международной космической станции по крайней мере до 2030 года, заявил глава NASA Билл Нельсон, его цитирует РИА «Новости».

«У нас есть все основания полагать, что русские будут [продолжать работу по] космической станции в ближайшем будущем», — сказал он на слушаниях в конгрессе.

Нельсон также отметил, что организация надеется на продолжение сотрудничества по программе до 2030 года и что его слова основываются на существующих между NASA и «Роскосмосом» «профессиональных отношений». Он также указал, что информация по поводу выхода России из программы МКС неверна.

Ранее глава «Роскосмоса» Дмитрий Рогозин сообщил, что Россия определилась со сроками завершения работы на МКС. Он не назвал сроки, но уточнил, что РФ продолжает работу на орбитальной станции до 2024 года. По мнению эксперта Института космических исследований РАН Натан Эйсмонт заявил, что эксплуатация МКС станет практически невозможной, если Россия выйдет из программы.
'''

In [105]:
text_vec = tfidf_vectorizer.transform([econ_text])
km.predict(text_vec)

array([0], dtype=int32)